# Import KEGG Pathways from Web Service

This example demonstrates how to use external web services with Cytoscape.  As an example, we will use [KEGG REST API](http://www.kegg.jp/kegg/docs/keggapi.html).

## Requirments
In addition to basic cyREST setup, you need to install the following Cytoscpae App to run this workflow:

* [KEGGScape 0.7.x](http://apps.cytoscape.org/apps/keggscape)


## Input and Output

* Input - Disease name
* Output - Cytoscape session file containing all KEGG pathways known to be related to the disease.

In [7]:
import requests
import json
import pandas as pd
import io
from IPython.display import Image

# Basic Setup
PORT_NUMBER = 1234
BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'

# KEGG API URL
KEGG_API_URL = 'http://rest.kegg.jp/'

# Header for posting data to the server as JSON
HEADERS = {'Content-Type': 'application/json'}

# Delete all networks in current session
requests.delete(BASE + 'session')

<Response [200]>

## Get list of entries from KEGG Disease database about cancer

In [8]:
# Find information about cancer from KEGG disease database.
query = 'cancer'

res = requests.get(KEGG_API_URL + '/find/disease/' + query)
pathway_list = res.content.decode('utf8')

disease_df = pd.read_csv(io.StringIO(pathway_list), delimiter='\t',  header=None, names=['id', 'name'])
disease_df

,id,name
0,ds:H00013,Small cell lung cancer
1,ds:H00014,Non-small cell lung cancer
2,ds:H00016,Oral cancer
3,ds:H00017,Esophageal cancer
4,ds:H00018,Gastric cancer
5,ds:H00019,Pancreatic cancer
6,ds:H00020,Colorectal cancer
7,ds:H00022,Bladder cancer
8,ds:H00023,Testicular cancer
9,ds:H00024,Prostate cancer


## Piping the result to KEGG Pathway database to get list of related pathways

In [9]:
disease_ids = disease_df['id']
disease_urls = disease_ids.apply(lambda x: KEGG_API_URL + 'get/' + x)

def disease_parser(entry):
    lines = entry.split('\n')
    data = {}
    
    last_key = None
    for line in lines:
        if '///' in line:
            return data
        
        parts = line.split(' ')
        if parts[0] is not None and len(parts[0]) != 0:
            last_key = parts[0]
            data[parts[0]] = line.replace(parts[0], '').strip()
        else:
            last_val = data[last_key]
            data[last_key] = last_val + '|' + line.strip()
    return data   

result = []
for url in disease_urls:
        res = requests.get(url)
        rows = disease_parser(res.content)
        result.append(rows)

In [10]:
disease_df = pd.DataFrame(result)
pathways = disease_df['PATHWAY'].dropna().unique()

p_urls = []
for pathway in pathways:
    entries = pathway.split('|')
    for en in entries:
        url = KEGG_API_URL + 'get/' + en.split(' ')[0].split('(')[0] + '/kgml'
        p_urls.append(url)

## Import all pathways directly into Cytoscape

In [11]:
def create_from_list(network_list):
    server_res = requests.post(BASE + 'networks?source=url&collection=' + query, data=json.dumps(network_list), headers=HEADERS)
    return server_res.json()

url_list = list(set(p_urls))
pathway_suids = create_from_list(url_list)

### Result
In your Cytoscape window, you can see the pathway diagrams as interactive Cytoscape view:

![](http://chianti.ucsd.edu/~kono/images/python_kegg_workflow.png)

## Embed the result as Cytoscape.js Widget
py2cytoscape has utility to visualize any Cytoscape network as an embedded widget.  You can do it with few lines of code:

In [12]:
from py2cytoscape import cytoscapejs as cyjs

res1 = requests.get(BASE + 'styles/KEGG Style.json')
kegg_style = res1.json()

# Load local network files
kegg_url = BASE + 'networks/' + str(pathway_suids[5]['networkSUID'][0])
net1 =  requests.get(kegg_url + '/views/first')

# Render it!
cyjs.render(net1.json(), style=kegg_style[0]['style'], background='white')

----
[Back to Index](../index.ipynb)
